# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [5]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [6]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [79]:
df = pd.DataFrame()
df["Quantity"] = data.copy().groupby(["CustomerID","ProductName"]).sum()["Quantity"]
df.reset_index(inplace=True)
df.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [80]:
df2 = pd.pivot_table(products,values="Quantity", index="ProductName", columns="CustomerID", fill_value=0)
df2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [81]:
df3 = pd.DataFrame(1/(1 + squareform(pdist(df2.T, 'euclidean'))), 
                         index=df2.columns, columns=df2.columns)

df3.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [83]:
df4 = df3.copy().reset_index()[["CustomerID",264]].sort_values(264, ascending=False)
df5 = df4[1:].head()
df5

CustomerID,CustomerID,264
14,1008,0.088152
33,2617,0.088152
17,1072,0.087410
0,33,0.087047
47,3535,0.086333


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [105]:
lista264 = list(df5["CustomerID"])
df5=df[df["CustomerID"].isin(lista264)]
df5.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [106]:
df6 = pd.DataFrame()
df6["Quantity"]=df5.copy().groupby(["ProductName"]).sum()["Quantity"]
df6.reset_index(inplace=True)
df6.sort_values("Quantity", ascending=False, inplace=True)
df6.head()

,ProductName,Quantity
177,Soupfoamcont12oz 112con,3
218,Wine - Two Oceans Cabernet,3
36,Bread - Italian Roll With Herbs,3
199,"Veal - Inside, Choice",3
148,Potatoes - Idaho 100 Count,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [107]:
df7 = df6.copy().merge(df2, on="ProductName")
df7_zero = df7.copy()[df7[264]==0][["ProductName", "Quantity"]]
df7_zero.head()

,ProductName,Quantity
0,Soupfoamcont12oz 112con,3
1,Wine - Two Oceans Cabernet,3
2,Bread - Italian Roll With Herbs,3
3,"Veal - Inside, Choice",3
7,"Fish - Scallops, Cold Smoked",3


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [118]:
recommend = {}
customers_unique=list(set(df.copy()["CustomerID"]))

def recoCust(_id):
    df4_r = df3.copy().reset_index()[["CustomerID",_id]].sort_values(_id, ascending=False)
    df5_r = df4_r[1:].head()
    
    lista_r = list(df5_r["CustomerID"])
    df5_r=df[df["CustomerID"].isin(lista_r)]
    
    df6_r = pd.DataFrame()
    df6_r["Quantity"]=df5_r.copy().groupby(["ProductName"]).sum()["Quantity"]
    df6_r.reset_index(inplace=True)
    df6_r.sort_values("Quantity", ascending=False, inplace=True)
    
    df7_r = df6_r.copy().merge(df2, on="ProductName")
    df7_zero_r = df7_r.copy()[df7_r[_id]==0]["ProductName"]
    recommend_list=list(df7_zero_r[:5])
    
    return recommend_list
    
recommend={i: recoCust(i) for i in customers_unique}

In [120]:
recommend[264]

['Soupfoamcont12oz 112con',
 'Wine - Two Oceans Cabernet',
 'Bread - Italian Roll With Herbs',
 'Veal - Inside, Choice',
 'Fish - Scallops, Cold Smoked']

In [121]:
len(list(customers_unique))

1000

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [125]:
cust_df = pd.DataFrame(recommend).T
cust_df.reset_index(inplace=True)
cust_df.columns=["CustomerID","Top1", "Top2", "Top3", "Top4", "Top5"]
cust_df.head()

,CustomerID,Top1,Top2,Top3,Top4,Top5
0,83973,Cocoa Butter,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Chicken - Wieners
1,49159,"Pasta - Penne, Rigate, Dry",Cookies - Assorted,Puree - Mocha,Cheese - Parmesan Grated,Cheese - Victor Et Berthold
2,22536,Bacardi Breezer - Tropical,Cheese - Mozzarella,Cheese Cloth No 100,Bread - Italian Roll With Herbs,Bread - Hot Dog Buns
3,18441,Soupfoamcont12oz 112con,Anchovy Paste - 56 G Tube,Sprouts - Baby Pea Tendrils,"Lemonade - Natural, 591 Ml",Sausage - Breakfast
4,59399,"Shrimp - Baby, Warm Water",Knife Plastic - White,Towels - Paper / Kraft,Nantucket - Pomegranate Pear,Snapple - Iced Tea Peach


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [126]:
dff = pd.DataFrame(1/(1 + squareform(pdist(df2.T, 'cosine'))), 
                         index=df2.columns, columns=df2.columns)

dff.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.530003,0.563492,0.543546,0.542393,0.541054,0.522016,0.530783,0.533096,0.535293,...,0.530306,0.517987,0.527662,0.511471,0.534181,0.513809,0.528795,0.520208,0.529481,0.527142
200,0.530003,1.000000,0.537166,0.535098,0.521226,0.519193,0.556945,0.516729,0.536350,0.541451,...,0.545224,0.533067,0.555246,0.543554,0.530308,0.521925,0.573177,0.537284,0.526167,0.527280
264,0.563492,0.537166,1.000000,0.531963,0.550168,0.525368,0.538967,0.514932,0.537046,0.524193,...,0.546880,0.525550,0.535062,0.531806,0.553085,0.560225,0.548538,0.538340,0.554929,0.527142
356,0.543546,0.535098,0.531963,1.000000,0.525982,0.523986,0.536792,0.540890,0.516901,0.519482,...,0.536321,0.520565,0.533119,0.534066,0.528874,0.530406,0.520138,0.536202,0.544727,0.535974
412,0.542393,0.521226,0.550168,0.525982,1.000000,0.534073,0.542308,0.547743,0.530459,0.543050,...,0.527903,0.565369,0.528847,0.529278,0.541674,0.536686,0.519628,0.538710,0.546943,0.552636


In [ ]:
recommend_c = {}
customers_unique=list(set(df.copy()["CustomerID"]))

def recoCosine(_id):
    df4_r = dff.copy().reset_index()[["CustomerID",_id]].sort_values(_id, ascending=False)
    df5_r = df4_r[1:].head()
    
    lista_r = list(df5_r["CustomerID"])
    df5_r=df[df["CustomerID"].isin(lista_r)]
    
    df6_r = pd.DataFrame()
    df6_r["Quantity"]=df5_r.copy().groupby(["ProductName"]).sum()["Quantity"]
    df6_r.reset_index(inplace=True)
    df6_r.sort_values("Quantity", ascending=False, inplace=True)
    
    df7_r = df6_r.copy().merge(df2, on="ProductName")
    df7_zero_r = df7_r.copy()[df7_r[_id]==0]["ProductName"]
    recommend_list=list(df7_zero_r[:5])
    
    return recommend_list
    
recommend_c={i: recoCosine(i) for i in customers_unique}

In [ ]:
recommend_c[264]